In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests

In [6]:
# Load our growth rates data
growth_data = pd.read_csv('../01_MergedData/data/top_counties.csv')

# Rename '2022' column to 'Income Growth'
growth_data = growth_data.rename(columns={'2022': 'Income Growth'})

# Function to fetch FEMA disaster declarations data
def fetch_fema_data(start_date, end_date):
    url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
    params = {
        "$filter": f"declarationDate ge '{start_date}' and declarationDate le '{end_date}'",
        "$format": "json"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return pd.DataFrame(response.json()['DisasterDeclarationsSummaries'])
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

# Fetch FEMA data for the last 10 years
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=3650)).strftime('%Y-%m-%d')
fema_data = fetch_fema_data(start_date, end_date)

# Convert declarationDate to datetime
fema_data['declarationDate'] = pd.to_datetime(fema_data['declarationDate'])

# Prepare growth data
growth_data[['County', 'State']] = growth_data['GeoName'].str.split(', ', expand=True)
growth_data['County'] = growth_data['County'].str.replace(' County', '')

print("Growth data counties:")
print(growth_data[['County', 'State']])

print("\nFEMA data sample:")
print(fema_data[['designatedArea', 'state']].head())

# Filter FEMA data for our top counties
top_counties_fema = fema_data[fema_data.apply(lambda row: any((row['designatedArea'] == county and row['state'] == state) 
                                                              for county, state in zip(growth_data['County'], growth_data['State'])), axis=1)]

print(f"\nNumber of matching disaster records: {len(top_counties_fema)}")

if len(top_counties_fema) == 0:
    print("No matches found between growth data counties and FEMA disaster data.")
    print("Please check if the county names and state abbreviations match exactly.")
else:
    # Group by county and date, count disasters
    disaster_counts = top_counties_fema.groupby(['state', 'designatedArea', 'declarationDate']).size().reset_index(name='count')

    # Pivot the data to create a time series for each county
    pivot_data = disaster_counts.pivot_table(values='count', index='declarationDate', 
                                             columns=['state', 'designatedArea'], fill_value=0)

    # Resample to monthly frequency and calculate cumulative sum
    monthly_data = pivot_data.resample('M').sum().cumsum()

    # Plot the time series
    plt.figure(figsize=(20, 10))
    for column in monthly_data.columns:
        plt.plot(monthly_data.index, monthly_data[column], label=f"{column[1]}, {column[0]}")

    plt.title("Cumulative Number of Disasters by County Over Time (Top Counties)")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Number of Disasters")
    plt.legend(title="County, State", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    # plt.savefig("top_counties_disaster_time_series.png")
    # plt.close()
    plt.show()

    # print("Time series plot saved as 'top_counties_disaster_time_series.png'")

    # Print summary statistics
    print("\nDisaster Counts for Top Counties:")
    total_disasters = monthly_data.iloc[-1].sort_values(ascending=False)
    for (state, county), count in total_disasters.items():
        print(f"{county}, {state}: {count}")

    # Merge disaster counts with growth data
    growth_data['DisasterCount'] = growth_data.apply(lambda row: total_disasters.get((row['State'], row['County']), 0), axis=1)

    # Display combined results
    print("\nTop Counties with Growth Rates and Disaster Counts:")
    print(growth_data[['GeoName', 'Income Growth', 'CAGR', 'Total Growth Rate', 'DisasterCount']])

    # Save the combined data
    growth_data.to_csv("top_counties_growth_and_disasters.csv", index=False)
    print("\nCombined data saved as 'top_counties_growth_and_disasters.csv'")

print("\nScript execution completed.")

Growth data counties:
          County State
0         Denver    CO
1       Gallatin    MT
2        Collier    FL
3      Deschutes    OR
4           Utah    UT
5         Martin    FL
6         Benton    AR
7     Palm Beach    FL
8   Indian River    FL
9        Boulder    CO
10    Washington    UT
11         Pinal    AZ
12        Travis    TX
13      Kootenai    ID
14          Hall    GA
15      Sarasota    FL
16     Salt Lake    UT
17        Canyon    ID
18      Missoula    MT
19    Charleston    SC
20      Davidson    TN
21       Flagler    FL
22    Williamson    TN

FEMA data sample:
                             designatedArea state
0  Washington (County)(in (P)MSA 5520,6480)    RI
1                       Washington (County)    KS
2                             Baca (County)    CO
3                            Kiowa (County)    CO
4                          Prowers (County)    CO

Number of matching disaster records: 0
No matches found between growth data counties and FEMA disaster dat